<a href="https://colab.research.google.com/github/wmalevich/ds_course/blob/main/hw_4(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Комбинированные (Hybrid) Алгоритмы.
"""
Мы создаём простую среду с определенным количеством состояний и действий.
Метод step в классе SimpleEnv определяет, как изменяется состояние и какая награда выдается в зависимости от действия агента.
Метод reset сбрасывает среду в начальное состояние.
Функция hybrid_policy определяет гибридную политику: если для текущего состояния в Q-таблице есть положительные значения,
выбирается действие с максимальным значением; в противном случае действие выбирается случайно.
Функция train обучает агента, обновляя Q-таблицу на основе взаимодействия с средой.
После обучения мы выводим Q-таблицу, которая показывает, какое действие является наилучшим для каждого состояния с точки зрения ожидаемой награды.
"""

import numpy as np
import random

class SimpleEnv:
    def __init__(self):
        # Определяем размер пространства состояний и действий
        self.state_space_size = 4  # Например, 4 различных состояния
        self.action_space_size = 2  # Например, 2 действия (0 и 1)

    def step(self, state, action):
        # Метод, определяющий логику изменения состояния и назначения награды
        # Пример: состояние циклически увеличивается и обнуляется
        next_state = (state + 1) % self.state_space_size
        # Награда зависит от выбранного действия
        reward = 1 if action == 0 else -1
        # Проверяем, завершился ли эпизод (для упрощения считаем, что после каждого действия эпизод завершается)
        done = next_state == 0
        return next_state, reward, done

    def reset(self):
        # Возвращает начальное состояние среды
        return 0

def hybrid_policy(q_table, state):
    # Гибридная политика: используем Q-таблицу, если есть положительные значения, иначе случайный выбор
    if np.max(q_table[state]) > 0:
        return np.argmax(q_table[state])
    else:
        return random.choice([0, 1])

def train(env, episodes=1000, learning_rate=0.8, gamma=0.95):
    # Инициализация Q-таблицы нулями
    q_table = np.zeros((env.state_space_size, env.action_space_size))

    for episode in range(episodes):
        state = env.reset()
        done = False

        while not done:
            action = hybrid_policy(q_table, state)  # Выбор действия согласно гибридной политике
            next_state, reward, done = env.step(state, action)

            # Обновление Q-таблицы
            q_table[state, action] = (1 - learning_rate) * q_table[state, action] + \
                                     learning_rate * (reward + gamma * np.max(q_table[next_state]))
            state = next_state

    return q_table

# Создаем экземпляр среды и обучаем агента
env = SimpleEnv()
q_table = train(env)

# Выводим обученную Q-таблицу
print("Обученная Q-таблица:")
print(q_table)

Обученная Q-таблица:
[[ 9.75706729  0.        ]
 [ 0.          9.21796556]
 [10.75575323 -0.8       ]
 [10.26921392  0.        ]]


In [ ]:
from sklearn.base import BaseEstimator
from sklearn.model_selection import RandomizedSearchCV
import numpy as np
import random

class QLearningWrapper(BaseEstimator):
    def __init__(self, env, episodes=1000, learning_rate=0.8, gamma=0.95):
        self.env = env
        self.episodes = episodes
        self.learning_rate = learning_rate
        self.gamma = gamma

    def fit(self, X=None, y=None):
        q_table = np.zeros((self.env.state_space_size, self.env.action_space_size))

        for episode in range(self.episodes):
            state = self.env.reset()
            done = False

            while not done:
                action = hybrid_policy(q_table, state)  # Выбор действия согласно гибридной политике
                next_state, reward, done = self.env.step(state, action)

                # Обновление Q-таблицы
                q_table[state, action] = (1 - self.learning_rate) * q_table[state, action] + \
                                         self.learning_rate * (reward + self.gamma * np.max(q_table[next_state]))
                state = next_state

        self.q_table_ = q_table
        return self

    def score(self, X=None, y=None):
        return 0

# Создаем экземпляр среды
env = SimpleEnv()

# Определяем диапазоны для подбора гиперпараметров
param_dist = {
    'episodes': [1000, 2000],
    'learning_rate': np.arange(0.1, 1.0, 0.1),
    'gamma': np.arange(0.1, 1.0, 0.1)
}

# Создаем пустые массивы для данных X и y
X_dummy = np.empty((10, env.state_space_size))
y_dummy = np.empty(10)

random_search = RandomizedSearchCV(estimator=QLearningWrapper(env), param_distributions=param_dist, cv=3, n_iter=10)

random_search.fit(X=X_dummy, y=y_dummy)

best_params = random_search.best_params_
print("Лучшие параметры:", best_params)

# Получаем обученную модель с лучшими параметрами
best_q_table = random_search.best_estimator_

# Выводим обученную Q-таблицу
print("Обученная Q-таблица с лучшими параметрами:")
print(best_q_table.q_table_)

Лучшие параметры: {'learning_rate': 0.9, 'gamma': 0.8, 'episodes': 1000}
Обученная Q-таблица с лучшими параметрами:
[[ 2.83197832  0.        ]
 [ 2.2899729  -0.9       ]
 [ 0.          1.61246612]
 [ 3.26558266 -0.252     ]]
